# DAMR
Hi, here is the code of DAMR(Dynamic Ajacency Matrix Representation) 
for data imputation in COVID-19 dataset.

# 1. Import library

In [36]:
#1.Import library
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import bipartite
import copy
from sklearn.preprocessing import StandardScaler
import scipy.special as special
import scipy.sparse as sp
import pickle
import os
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import tensorflow as tf
#import tensorflow_addons as tfa
import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, GRU, Input
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dropout, Dense, LeakyReLU, GRU, Concatenate, Reshape, Softmax, Attention
from tensorflow.keras import activations
from tensorflow.keras.models import Model
#from keras.layers import LeakyReLU
from spektral.layers import  GCSConv, DiffusionConv, GATConv, ARMAConv, GCNConv
from tensorflow.keras.regularizers import l2
#from my own library import Generate_missing and other models
from model import * #generate_miss(data,perc),output:data after generate missing.
import FancyImpute
from FancyImpute import *
import torch
#Bidirectional LSTM
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from BiLSTM import BiLSTM_3D

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 2. Data Pre-processing of COVID dataset

In [47]:
#Define hyperparameter
Miss_perc=0.3
Split_perc1=0.8
Split_perc2=0.1
#

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [50]:
#Generate Missing 3D.
path='Data/INDIA/raw'
COVID_features=Csv2Tensor(path)
COVID_features=COVID_features[:,:,:]
COVID_features=MinMax3D(COVID_features)#(503,3,7)
COVID_features_train=COVID_features[:int(Split_perc1*COVID_features.shape[0]),:,:]
print(COVID_features_train.shape)#(402, 3, 7)
COVID_features_val=COVID_features[int(Split_perc1*COVID_features.shape[0]):int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]),:,:]
print(COVID_features_val.shape)#(50, 3, 7)
COVID_features_test=COVID_features[int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]):,:,:]
print(COVID_features_test.shape)#(51, 3, 7)

#The mask and missing data has been created with a given miss percentage.
#It will be used for baseline and for DAMR
COVID_features_train_miss,COVID_features_train_mask=generate_miss_3D(COVID_features_train,Miss_perc)
COVID_features_val_miss,COVID_features_val_mask=generate_miss_3D(COVID_features_val,Miss_perc)
COVID_features_test_miss,COVID_features_test_mask=generate_miss_3D(COVID_features_test,Miss_perc)
COVID_features_miss,COVID_features_mask=generate_miss_3D(COVID_features,Miss_perc)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


()
()
()
()
()
()
()
()
(84096, 4, 8)
(10512, 4, 8)
(10512, 4, 8)


In [ ]:
#Run baseline(KNN,MICE,MF,MEAN,SLIDING)#Return:RMSE,MAE,MAPE,MRE
#For the model of FancyImpute,we just need to test.
KNN_RMSE,KNN_MAE,KNN_MAPE,KNN_MRE=KNN_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
MICE_RMSE,MICE_MAE,MICE_MAPE,MICE_MRE=MICE_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
MF_RMSE,MF_MAE,MF_MAPE,MF_MRE=MF_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
MEAN_RMSE,MEAN_MAE,MEAN_MAPE,MEAN_MRE=MEAN_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
Sliding_RMSE,Sliding_MAE,Sliding_MAPE,Sliding_MRE,Sliding_out=Sliding_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask,32)
print("KNN_RMSE",KNN_RMSE,"KNN_MAE",KNN_MAE,"KNN_MAPE",KNN_MAPE,"KNN_MRE",KNN_MRE)
print("MICE_RMSE",MICE_RMSE,"MICE_MAE",MICE_MAE,"MICE_MAPE",MICE_MAPE,"MICE_MRE",MICE_MRE)
print("MF_RMSE",MF_RMSE,"MF_MAE",MF_MAE,"MF_MAPE",MF_MAPE,"MF_MRE",MF_MRE)
print("MEAN_RMSE",MEAN_RMSE,"MEAN_MAE",MEAN_MAE,"MEAN_MAPE",MEAN_MAPE,"MEAN_MRE",MEAN_MRE)
print("Sliding_RMSE",Sliding_RMSE,"Sliding_MAE",Sliding_MAE,"Sliding_MAPE",Sliding_MAPE,"Sliding_MRE",Sliding_MRE)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Imputing row 1/10512 with 1 missing, elapsed time: 14.055
Imputing row 101/10512 with 2 missing, elapsed time: 14.064
Imputing row 201/10512 with 0 missing, elapsed time: 14.071
Imputing row 301/10512 with 1 missing, elapsed time: 14.079
Imputing row 401/10512 with 1 missing, elapsed time: 14.089
Imputing row 501/10512 with 1 missing, elapsed time: 14.097
Imputing row 601/10512 with 1 missing, elapsed time: 14.105
Imputing row 701/10512 with 0 missing, elapsed time: 14.111
Imputing row 801/10512 with 2 missing, elapsed time: 14.120
Imputing row 901/10512 with 3 missing, elapsed time: 14.127
Imputing row 1001/10512 with 2 missing, elapsed time: 14.135
Imputing row 1101/10512 with 3 missing, elapsed time: 14.143
Imputing row 1201/10512 with 0 missing, elapsed time: 14.150
Imputing row 1301/10512 with 1 missing, elapsed time: 14.158
Imputing row 1401/10512 with 3 missing, elapsed time: 14.166
Imputing row 1501/10512 with 1 missing, elapsed time: 14.174
Imputing row 1601/10512 with 1 missi

Imputing row 3301/10512 with 2 missing, elapsed time: 15.140
Imputing row 3401/10512 with 0 missing, elapsed time: 15.152
Imputing row 3501/10512 with 0 missing, elapsed time: 15.165
Imputing row 3601/10512 with 2 missing, elapsed time: 15.178
Imputing row 3701/10512 with 2 missing, elapsed time: 15.190
Imputing row 3801/10512 with 2 missing, elapsed time: 15.201
Imputing row 3901/10512 with 1 missing, elapsed time: 15.213
Imputing row 4001/10512 with 0 missing, elapsed time: 15.228
Imputing row 4101/10512 with 2 missing, elapsed time: 15.242
Imputing row 4201/10512 with 1 missing, elapsed time: 15.256
Imputing row 4301/10512 with 2 missing, elapsed time: 15.270
Imputing row 4401/10512 with 2 missing, elapsed time: 15.285
Imputing row 4501/10512 with 2 missing, elapsed time: 15.301
Imputing row 4601/10512 with 1 missing, elapsed time: 15.311
Imputing row 4701/10512 with 1 missing, elapsed time: 15.319
Imputing row 4801/10512 with 3 missing, elapsed time: 15.328
Imputing row 4901/10512 

Imputing row 7301/10512 with 2 missing, elapsed time: 14.007
Imputing row 7401/10512 with 0 missing, elapsed time: 14.016
Imputing row 7501/10512 with 1 missing, elapsed time: 14.025
Imputing row 7601/10512 with 2 missing, elapsed time: 14.033
Imputing row 7701/10512 with 2 missing, elapsed time: 14.044
Imputing row 7801/10512 with 1 missing, elapsed time: 14.056
Imputing row 7901/10512 with 2 missing, elapsed time: 14.068
Imputing row 8001/10512 with 3 missing, elapsed time: 14.079
Imputing row 8101/10512 with 1 missing, elapsed time: 14.092
Imputing row 8201/10512 with 1 missing, elapsed time: 14.104
Imputing row 8301/10512 with 3 missing, elapsed time: 14.116
Imputing row 8401/10512 with 1 missing, elapsed time: 14.126
Imputing row 8501/10512 with 1 missing, elapsed time: 14.137
Imputing row 8601/10512 with 2 missing, elapsed time: 14.148
Imputing row 8701/10512 with 3 missing, elapsed time: 14.157
Imputing row 8801/10512 with 2 missing, elapsed time: 14.168
Imputing row 8901/10512 

Imputing row 1/10512 with 0 missing, elapsed time: 14.000
Imputing row 101/10512 with 2 missing, elapsed time: 14.009
Imputing row 201/10512 with 1 missing, elapsed time: 14.018
Imputing row 301/10512 with 0 missing, elapsed time: 14.026
Imputing row 401/10512 with 0 missing, elapsed time: 14.035
Imputing row 501/10512 with 2 missing, elapsed time: 14.044
Imputing row 601/10512 with 0 missing, elapsed time: 14.054
Imputing row 701/10512 with 3 missing, elapsed time: 14.063
Imputing row 801/10512 with 0 missing, elapsed time: 14.073
Imputing row 901/10512 with 1 missing, elapsed time: 14.082
Imputing row 1001/10512 with 0 missing, elapsed time: 14.090
Imputing row 1101/10512 with 0 missing, elapsed time: 14.099
Imputing row 1201/10512 with 0 missing, elapsed time: 14.107
Imputing row 1301/10512 with 1 missing, elapsed time: 14.114
Imputing row 1401/10512 with 0 missing, elapsed time: 14.123
Imputing row 1501/10512 with 2 missing, elapsed time: 14.131
Imputing row 1601/10512 with 0 missi

Imputing row 3901/10512 with 2 missing, elapsed time: 15.762
Imputing row 4001/10512 with 0 missing, elapsed time: 15.778
Imputing row 4101/10512 with 2 missing, elapsed time: 15.790
Imputing row 4201/10512 with 2 missing, elapsed time: 15.806
Imputing row 4301/10512 with 1 missing, elapsed time: 15.821
Imputing row 4401/10512 with 0 missing, elapsed time: 15.837
Imputing row 4501/10512 with 2 missing, elapsed time: 15.853
Imputing row 4601/10512 with 2 missing, elapsed time: 15.864
Imputing row 4701/10512 with 1 missing, elapsed time: 15.880
Imputing row 4801/10512 with 0 missing, elapsed time: 15.900
Imputing row 4901/10512 with 2 missing, elapsed time: 15.929
Imputing row 5001/10512 with 1 missing, elapsed time: 15.941
Imputing row 5101/10512 with 2 missing, elapsed time: 15.955
Imputing row 5201/10512 with 1 missing, elapsed time: 15.975
Imputing row 5301/10512 with 1 missing, elapsed time: 15.989
Imputing row 5401/10512 with 2 missing, elapsed time: 16.008
Imputing row 5501/10512 

Imputing row 7101/10512 with 2 missing, elapsed time: 14.998
Imputing row 7201/10512 with 1 missing, elapsed time: 15.006
Imputing row 7301/10512 with 2 missing, elapsed time: 15.015
Imputing row 7401/10512 with 1 missing, elapsed time: 15.025
Imputing row 7501/10512 with 2 missing, elapsed time: 15.035
Imputing row 7601/10512 with 1 missing, elapsed time: 15.044
Imputing row 7701/10512 with 2 missing, elapsed time: 15.052
Imputing row 7801/10512 with 1 missing, elapsed time: 15.059
Imputing row 7901/10512 with 2 missing, elapsed time: 15.068
Imputing row 8001/10512 with 1 missing, elapsed time: 15.076
Imputing row 8101/10512 with 0 missing, elapsed time: 15.088
Imputing row 8201/10512 with 1 missing, elapsed time: 15.100
Imputing row 8301/10512 with 3 missing, elapsed time: 15.109
Imputing row 8401/10512 with 0 missing, elapsed time: 15.117
Imputing row 8501/10512 with 2 missing, elapsed time: 15.125
Imputing row 8601/10512 with 1 missing, elapsed time: 15.134
Imputing row 8701/10512 

/opt/anaconda3/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


                                     CVXPY                                     
                                     v1.2.0                                    
(CVXPY) May 29 09:49:50 PM: Your problem has 42048 variables, 1 constraints, and 0 parameters.
(CVXPY) May 29 09:49:50 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 29 09:49:50 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 29 09:49:50 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 29 09:49:50 PM: Compiling problem (target solver=CVXOPT).
(CVXPY) May 29 09:49:50 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffi

# Run Baseline

In [6]:
##Lags for variable 1 ,modify input variable here!
COVID_X1_all=COVID_features[:,0,:]
#Transform the shape into Bidirectional problems
COVID_X1_all=series_to_supervised(COVID_X1_all,15,16)
COVID_Y1_all=COVID_X1_all.iloc[:,15*7:15*7+7]
COVID_X1_all_1=COVID_X1_all.iloc[:,:15*7]
COVID_X1_all_2=COVID_X1_all.iloc[:,15*7+7:]
COVID_X1_all=np.hstack((COVID_X1_all_1,COVID_X1_all_2))
COVID_X1_all=np.array(COVID_X1_all)
COVID_X1_all=np.reshape(COVID_X1_all,(len(COVID_X1_all),COVID_features.shape[2],30))
#Input 1->Groundtruth
COVID_features=COVID_features[15:-15,:,:]#(473, 3, 7)
print(COVID_features.shape)
#Input2->Groundtruth
print(COVID_X1_all.shape)#(473,7,30)
COVID_Y1_all=np.array(COVID_Y1_all)
COVID_Y1_all=np.reshape(COVID_Y1_all,(len(COVID_Y1_all),7,1))
#Input3->Groundtruth
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
#Output1->Groundtruth
print(COVID_Y1_all.shape)#(473, 7, 1)

(473, 3, 7)
(473, 7, 30)
(473, 7, 1)


In [7]:
##Input and output in DAMR!
# print(COVID_features_aftslid.shape)#473*3*7
# print(COVID_X1_all_aftslid.shape)#473*7*30
# print(COVID_Y1_all.shape)#(473, 7, 1)
# print(Adj_dist.shape)#7*7

#The mask and missing data has been created with a given miss percentage.
#It will be used for baseline and for DAMR
#Generate Missing 3D.
COVID_features=Csv2Tensor(path)
COVID_features=COVID_features[:503,:,:]
COVID_features=MinMax3D(COVID_features)#(503,3,7)->Groundtruth
COVID_features_train=COVID_features[:int(Split_perc1*COVID_features.shape[0]),:,:]
print(COVID_features_train.shape)#(402, 3, 7)
COVID_features_val=COVID_features[int(Split_perc1*COVID_features.shape[0]):int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]),:,:]
print(COVID_features_val.shape)#(50, 3, 7)
COVID_features_test=COVID_features[int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]):,:,:]
print(COVID_features_test.shape)#(51, 3, 7)

#
COVID_features_train_miss,COVID_features_train_mask=generate_miss_3D(COVID_features_train,Miss_perc)#(402, 3, 7)
COVID_features_val_miss,COVID_features_val_mask=generate_miss_3D(COVID_features_val,Miss_perc)#(50, 3, 7)
COVID_features_test_miss,COVID_features_test_mask=generate_miss_3D(COVID_features_test,Miss_perc)#(51, 3, 7)
COVID_features_miss,COVID_features_mask=generate_miss_3D(COVID_features,Miss_perc)

Sliding_RMSE,Sliding_MAE,Sliding_MAPE,Sliding_MRE,Sliding_out=Sliding_impute_3D(COVID_features_miss,COVID_features,COVID_features_mask,6)#(503,3,7)




()
()
()
()
()
()
()
(402, 3, 7)
(50, 3, 7)
(51, 3, 7)


In [8]:
#Create lags for the first variable.
#Extract the first variable
lags=30
var=0
Sliding_out_var1=Sliding_out[:,var,:]
Sliding_out_var1_sup=series_to_supervised(Sliding_out_var1,lags,1)
Sliding_out_var1_sup=Sliding_out_var1_sup.iloc[:,:lags*Sliding_out.shape[2]]
print(Sliding_out_var1_sup)
#Input1: 473*7*30
COVID_X1_all_aftslid=np.array(Sliding_out_var1_sup).reshape(len(Sliding_out_var1_sup),COVID_features.shape[2],lags)
#Input2: 473*3*7
COVID_features_aftslid=Sliding_out[30:,:,:]
#Input3: Adj: 473*7*7(Initialization)
Adj_all = np.arange(len(COVID_features_aftslid)*COVID_features_aftslid.shape[2]*COVID_features_aftslid.shape[2]).reshape(len(COVID_features_aftslid),COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])#(473,7,7)
#Output:Groundtruth of the first variable->
COVID_Y1_all=COVID_features[30:,:,:]
COVID_Y1_all=COVID_Y1_all[:,var,:]
COVID_Y1_all=COVID_Y1_all.reshape(COVID_Y1_all.shape[0],COVID_Y1_all.shape[1],1)
#
print(COVID_X1_all_aftslid.shape,COVID_features_aftslid.shape,Adj_all.shape,COVID_Y1_all.shape)

     var1(t-30)  var2(t-30)  var3(t-30)  var4(t-30)  var5(t-30)  var6(t-30)  \
33    -1.000000   -0.999999   -1.000000   -0.999919   -0.999985   -0.999838   
34    -0.999896   -0.999999   -1.000000   -0.999893   -1.000000   -0.999804   
35    -0.999920   -0.999999   -1.000000   -0.999844   -0.999969   -0.999804   
36    -0.999837   -0.999999   -1.000000   -0.999807   -0.999985   -0.999411   
37    -0.999773   -0.999998   -1.000000   -0.999676   -0.999985   -0.999772   
..          ...         ...         ...         ...         ...         ...   
498    0.924809    0.902659    0.925887    0.736388    0.841088    0.517751   
499    0.926276    0.907211    0.915474    0.736629    0.844500    0.521352   
500    0.927524    0.911119    0.935666    0.739640    0.844483    0.515771   
501    0.926564    0.915140    0.923037    0.739608    0.843928    0.517279   
502    0.927259    0.903258    0.923942    0.742208    0.845661    0.518125   

     var7(t-30)  var1(t-29)  var2(t-29)  var3(t-29)

In [9]:
type(Sliding_out_var1_sup)

pandas.core.frame.DataFrame

# 3.Dynamic Adjacency matrix layer

In [10]:
#3.1 Distance-based adjacency matrix 
#Country: BZ CL ID IR MX PR SA.
COVID_lat=pd.DataFrame([-14.235,-35.6751,20.5937,32.4279,23.6345,-9.19,30.5595])
COVID_long=pd.DataFrame([-51.9253,-71.543,78.9629,53.688,-102.5528,-75.0152,22.9375])
Adj_dist=adj_dist(COVID_lat,COVID_long)#7*7

In [11]:
data1=pd.DataFrame([[-51.9253,-71.543,78.9629],[53.688,-102.5528,-75.0152]])
data2=pd.DataFrame([[-31.9253,-41.543,38.9629],[43.688,62.5528,65.0152]])
Global_MeanMI(data1,data2,2)
data1=pd.DataFrame(data1)
data2=pd.DataFrame(data2)
data1=np.array(data1).reshape(-1)
data2=np.array(data2).reshape(-1)

In [12]:
#3.2 Mean MI-based adjacency matrix
print(COVID_features_aftslid.shape)
MeanMI=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        MeanMI.append(Global_MeanMI(COVID_features_aftslid[:,:,i],
                                    COVID_features_aftslid[:,:,j],2))
print(np.array(MeanMI).shape)
MeanMI=np.array(MeanMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])
print(MeanMI.shape)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled1 = scaler.fit_transform(MeanMI)
MeanMI=scaled1#shape:7*7
print(MeanMI)

(473, 3, 7)
(49,)
(7, 7)
[[1.         0.07478726 0.6952845  0.70215367 0.53726317 0.88706082
  0.21050467]
 [0.         1.         0.         0.         0.         0.
  0.        ]
 [0.67768744 0.18237348 1.         0.50187347 0.66583975 0.65075892
  0.35425559]
 [0.70756421 0.         0.48603047 1.         0.36974813 0.75681556
  0.23931171]
 [0.53398215 0.2339395  0.65152798 0.412288   1.         0.49761157
  0.15670581]
 [0.89225742 0.05791869 0.66605478 0.75089298 0.49093116 1.
  0.18036188]
 [0.29727322 0.32679017 0.37802357 0.33791865 0.20166573 0.28142722
  1.        ]]


In [13]:
#3.3 CONCAT MI-based adjacency matrix
print(COVID_features_aftslid.shape)
ConcatMI=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        ConcatMI.append(Global_MeanMI(COVID_features_aftslid[:,:,i],
                                    COVID_features_aftslid[:,:,j],2))
ConcatMI=np.array(ConcatMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])
scaler = MinMaxScaler(feature_range=(0, 1))
scaled1 = scaler.fit_transform(ConcatMI)
ConcatMI=scaled1#shape:7*7
print(ConcatMI)

(473, 3, 7)
[[1.         0.07478726 0.6952845  0.70215367 0.53726317 0.88706082
  0.21050467]
 [0.         1.         0.         0.         0.         0.
  0.        ]
 [0.67768744 0.18237348 1.         0.50187347 0.66583975 0.65075892
  0.35425559]
 [0.70756421 0.         0.48603047 1.         0.36974813 0.75681556
  0.23931171]
 [0.53398215 0.2339395  0.65152798 0.412288   1.         0.49761157
  0.15670581]
 [0.89225742 0.05791869 0.66605478 0.75089298 0.49093116 1.
  0.18036188]
 [0.29727322 0.32679017 0.37802357 0.33791865 0.20166573 0.28142722
  1.        ]]


In [14]:
#3.4 Single Feature MI-based adjacency matrix
COVID_features_aftslid.shape
FeatureMI=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        FeatureMI.append(Global_EachMI(COVID_features_aftslid[:,:,i],
                                    COVID_features_aftslid[:,:,j],2))
#For the first feature,if choose the second feature, modify here!!
FeatureMI=np.array(FeatureMI)[:,2]
FeatureMI=np.array(FeatureMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])
scaler = MinMaxScaler(feature_range=(0, 1))
scaled1 = scaler.fit_transform(FeatureMI)
FeatureMI=scaled1#shape:7*7
#MeanMI,ConcatMI,FeatureMI

In [15]:
#3.5 Mean MI-based dynamic adjacency matrix
print(COVID_features_aftslid.shape)
#Index all of the rows and columns
DyMeanMI=[]
for k in range(COVID_features_aftslid.shape[0]):
    for i in range(COVID_features_aftslid.shape[2]):
        for j in range(COVID_features_aftslid.shape[2]):
            DyMeanMI.append(Global_MeanMI(COVID_features_aftslid[:k,:,i],
                                    COVID_features_aftslid[:k,:,j],2))
DyMeanMI=np.array(DyMeanMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[0])
DyMeanMI=DyMeanMI.transpose(2, 0, 1)
from fancyimpute import SimpleFill, NuclearNormMinimization, SoftImpute, BiScaler
for i in range(DyMeanMI.shape[0]):
    MEAN_imputer = SimpleFill()
    DyMeanMI[i,:,:]=MEAN_imputer.fit_transform(DyMeanMI[i,:,:])
print(DyMeanMI)#Here DyMeanMI is done

(473, 3, 7)


/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))


[[[0.55454292 0.72858623 0.59367175 ... 0.41634489 0.40073628 0.43037056]
  [0.59383422 0.39067769 0.28620198 ... 0.2942949  0.29894659 0.92909196]
  [0.25864293 0.64738681 0.34256545 ... 0.47784162 0.50089886 0.73647902]
  ...
  [0.64288089 0.31197601 0.65415232 ... 0.30240654 0.74372884 0.49611615]
  [0.38315602 0.73932323 0.99803072 ... 0.6997627  0.61457887 0.80609147]
  [0.85357178 0.68904613 0.29670291 ... 0.67865592 0.46717147 0.58526377]]

 [[0.46662446 0.16860787 0.89129346 ... 0.39323241 0.3201259  0.30172173]
  [0.43232222 0.37463529 0.59560094 ... 0.35705937 0.65544731 0.67737585]
  [0.61769984 0.90148675 0.49050462 ... 0.7256116  0.70050995 0.76796542]
  ...
  [0.30460784 0.50336008 0.51983271 ... 0.67103884 0.59397947 0.72760989]
  [0.54580424 0.72245353 0.68912637 ... 0.94336202 0.75396038 0.63409856]
  [0.61094105 0.79396237 0.55579258 ... 0.70436293 0.87345958 0.70777947]]

 [[0.52967197 0.05056747 0.36490559 ... 0.89381852 0.38277229 0.19997104]
  [0.49757813 0.289979

/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not mis

In [16]:
#3.6 Concat MI-based dynamic adjacency matrix
#Index all of the rows and columns
DyConcatMI=[]
for k in range(COVID_features_aftslid.shape[0]):
    for i in range(COVID_features_aftslid.shape[2]):
        for j in range(COVID_features_aftslid.shape[2]):
            DyConcatMI.append(Global_MeanMI(COVID_features_aftslid[:k,:,i],
                                    COVID_features_aftslid[:k,:,j],2))
DyConcatMI=np.array(DyConcatMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[0])
DyConcatMI=DyConcatMI.transpose(2, 0, 1)
from fancyimpute import SimpleFill, NuclearNormMinimization, SoftImpute, BiScaler
for i in range(DyConcatMI.shape[0]):
    MEAN_imputer = SimpleFill()
    DyConcatMI[i,:,:]=MEAN_imputer.fit_transform(DyConcatMI[i,:,:])
print(DyConcatMI)#Here DyConcatMI is done

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/

[[[0.55454292 0.72858623 0.59367175 ... 0.41634489 0.40073628 0.43037056]
  [0.59383422 0.39067769 0.28620198 ... 0.2942949  0.29894659 0.92909196]
  [0.25864293 0.64738681 0.34256545 ... 0.47784162 0.50089886 0.73647902]
  ...
  [0.64288089 0.31197601 0.65415232 ... 0.30240654 0.74372884 0.49611615]
  [0.38315602 0.73932323 0.99803072 ... 0.6997627  0.61457887 0.80609147]
  [0.85357178 0.68904613 0.29670291 ... 0.67865592 0.46717147 0.58526377]]

 [[0.46662446 0.16860787 0.89129346 ... 0.39323241 0.3201259  0.30172173]
  [0.43232222 0.37463529 0.59560094 ... 0.35705937 0.65544731 0.67737585]
  [0.61769984 0.90148675 0.49050462 ... 0.7256116  0.70050995 0.76796542]
  ...
  [0.30460784 0.50336008 0.51983271 ... 0.67103884 0.59397947 0.72760989]
  [0.54580424 0.72245353 0.68912637 ... 0.94336202 0.75396038 0.63409856]
  [0.61094105 0.79396237 0.55579258 ... 0.70436293 0.87345958 0.70777947]]

 [[0.52967197 0.05056747 0.36490559 ... 0.89381852 0.38277229 0.19997104]
  [0.49757813 0.289979

/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not mis

In [17]:
print(DyConcatMI.shape)

(473, 7, 7)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
#3.7 FFT-based sum batch's MI dynamic adjacency matrix
print(COVID_features_aftslid.shape)
T=[20,40,100];
Term1=T[0]
Term2=T[1]
Term3=T[2]
#Step1: Calculate Adj in each batch
#(1) Term1
a=[]
from math import ceil#Eg:Ceil(2.2)=3
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term1)):
            a.append(Global_MeanMI(COVID_features_aftslid[k*Term1:k*Term1+Term1,:,i],COVID_features_aftslid[k*Term1:k*Term1+Term1,:,j],2))
a=np.array(a).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term1))#a:(7, 7, 23)
a=a.transpose(2,0,1)
print(a.shape)#(24,7,7)

#(2) Term2
b=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term2)):
            b.append(Global_MeanMI(COVID_features_aftslid[k*Term2:k*Term2+Term2,:,i],COVID_features_aftslid[k*Term2:k*Term2+Term2,:,j],2))
b=np.array(b).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term2))#a:(7, 7, 23)
print(b.shape)#(7, 7, 12)
b=b.transpose(2,0,1)
print(b.shape)#(12,7,7)

#(3) Term3
c=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term3)):
            c.append(Global_MeanMI(COVID_features_aftslid[k*Term3:k*Term3+Term3,:,i],COVID_features_aftslid[k*Term3:k*Term3+Term3,:,j],2))
c=np.array(c).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term3))#a:(7, 7, 23)
print(c.shape)#(7, 7, 5)
c=c.transpose(2,0,1)
print(c.shape)#(5,7,7)
#Step2:Assign it to particular position:
#Each timestamp from the same batch share the same Adj.
#MeanMI
data=[]
for j in range(a.shape[0]):
    for i in range(Term1):
        data.append(a[j,:,:])
data=np.array(data)
test=data
DyConcatMI1=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI1.shape)#(473,7,7)

data=[]
for j in range(b.shape[0]):
    for i in range(Term2):
        data.append(b[j,:,:])
data=np.array(data)
test=data
DyConcatMI2=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI2.shape)#(473,7,7)

data=[]
for j in range(c.shape[0]):
    for i in range(Term3):
        data.append(c[j,:,:])
data=np.array(data)
test=data
DyConcatMI3=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI3.shape)#(473,7,7)

# Step3: Add all of the adjacency matrices
DyConcatFFTMI=[]
DyConcatFFTMI=DyConcatMI1+DyConcatMI2+DyConcatMI3
print(DyConcatFFTMI.shape)#(473,7,7)

# Step4: Normalize adjacency matrices
from sklearn.preprocessing import MinMaxScaler
for i in range(DyConcatFFTMI.shape[0]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    DyConcatFFTMI[i,:,:] = scaler.fit_transform(DyConcatFFTMI[i,:,:])
print(DyConcatFFTMI)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(473, 3, 7)
(24, 7, 7)
(7, 7, 12)
(12, 7, 7)
(7, 7, 5)
(5, 7, 7)
(473, 7, 7)
(473, 7, 7)
(473, 7, 7)
(473, 7, 7)
[[[1.         0.25800582 0.22132181 ... 0.32850394 0.11149538 0.35670379]
  [0.         1.         0.18491694 ... 0.         0.15972734 0.        ]
  [0.17680435 0.40193684 1.         ... 0.12119995 0.17652692 0.25527595]
  ...
  [0.167177   0.07727409 0.         ... 1.         0.20144598 0.34736167]
  [0.01141662 0.31566915 0.11807075 ... 0.25878442 1.         0.11841477]
  [0.22805771 0.         0.13821908 ... 0.37487229 0.         1.        ]]

 [[1.         0.25800582 0.22132181 ... 0.32850394 0.11149538 0.35670379]
  [0.         1.         0.18491694 ... 0.         0.15972734 0.        ]
  [0.17680435 0.40193684 1.         ... 0.12119995 0.17652692 0.25527595]
  ...
  [0.167177   0.07727409 0.         ... 1.         0.20144598 0.34736167]
  [0.01141662 0.31566915 0.11807075 ... 0.25878442 1.         0.11841477]
  [0.22805771 0.         0.13821908 ... 0.37487229 0.      

In [19]:
#3.8 FFT-based MI-based dynamic adjacency matrix
print(COVID_features_aftslid.shape)
T=[20,40,100];
Term1=T[0]
Term2=T[1]
Term3=T[2]

#Step1: Calculate Adj in each batch

#(1) Term1
a=[]
from math import ceil#Eg:Ceil(2.2)=3
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term1)):
            a.append(Global_MeanMI(COVID_features_aftslid[k*Term1:k*Term1+Term1,:,i],COVID_features_aftslid[k*Term1:k*Term1+Term1,:,j],2))
a=np.array(a).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term1))#a:(7, 7, 23)
a=a.transpose(2,0,1)
print(a.shape)#(24,7,7)

#(2) Term2
b=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term2)):
            b.append(Global_MeanMI(COVID_features_aftslid[k*Term2:k*Term2+Term2,:,i],COVID_features_aftslid[k*Term2:k*Term2+Term2,:,j],2))
b=np.array(b).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term2))#a:(7, 7, 23)
print(b.shape)#(7, 7, 12)
b=b.transpose(2,0,1)
print(b.shape)#(12,7,7)

#(3) Term3
c=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term3)):
            c.append(Global_MeanMI(COVID_features_aftslid[k*Term3:k*Term3+Term3,:,i],COVID_features_aftslid[k*Term3:k*Term3+Term3,:,j],2))
c=np.array(c).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term3))#a:(7, 7, 23)
print(c.shape)#(7, 7, 5)
c=c.transpose(2,0,1)
print(c.shape)#(5,7,7)
#Step2: Calculate MI's MI for periodic term 1
#Term1
temp=[]
for j in range(a.shape[0]):
    for i in range(a.shape[0]):
        temp.append(Global_MeanMI(a[j,:,:],a[i,:,:],2))#The first point MI's MI 
temp=np.array(temp).reshape(a.shape[0],a.shape[0])#24*24

#Step3: Weighted sum for the same periodic term
index_all=np.arange(temp.shape[0]*temp.shape[0]).reshape(a.shape[0],a.shape[0])#Initialization of index_all,shape=24*24
for i in range(temp.shape[0]):
    arrIndex=temp[i,:].argsort()
    temp[i,:]=temp[i,:][arrIndex[::-1]]
    index_all[i,:]=arrIndex
print(temp.shape)#24*24
print(index_all.shape)#24*24

all_term=5#Get the first 5 decomposition
a_all=np.zeros((a.shape[0],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2]))
for i in range(a.shape[0]):
    for u in range(all_term):
        a_all[i,:,:]=a_all[i,:,:]+a[i,:,:]+index_all[i,u]*temp[i,u]
print(a_all.shape)#(24,7,7)

#Step2: Calculate MI's MI for periodic term 2
#Term2
temp=[]
for j in range(b.shape[0]):
    for i in range(b.shape[0]):
        temp.append(Global_MeanMI(b[j,:,:],b[i,:,:],2))#The first point MI's MI 
temp=np.array(temp).reshape(b.shape[0],b.shape[0])#12*12

#Step3: Weighted sum for the same periodic term
index_all=np.arange(temp.shape[0]*temp.shape[0]).reshape(b.shape[0],b.shape[0])#Initialization of index_all,shape=24*24
for i in range(temp.shape[0]):
    arrIndex=temp[i,:].argsort()
    temp[i,:]=temp[i,:][arrIndex[::-1]]
    index_all[i,:]=arrIndex
print(temp.shape)#12*12
print(index_all.shape)#12*12

all_term=5#Get the first 5 decomposition
b_all=np.zeros((b.shape[0],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2]))
for i in range(b.shape[0]):
    for u in range(all_term):
        b_all[i,:,:]=b_all[i,:,:]+b[i,:,:]+index_all[i,u]*temp[i,u]
print(b_all.shape)#(12,7,7)

#Step2: Calculate MI's MI for periodic term 3
#Term3
temp=[]
for j in range(c.shape[0]):
    for i in range(c.shape[0]):
        temp.append(Global_MeanMI(c[j,:,:],c[i,:,:],2))#The first point MI's MI 
temp=np.array(temp).reshape(c.shape[0],c.shape[0])#12*12

#Step3: Weighted sum for the same periodic term
index_all=np.arange(temp.shape[0]*temp.shape[0]).reshape(c.shape[0],c.shape[0])#Initialization of index_all,shape=24*24
for i in range(temp.shape[0]):
    arrIndex=temp[i,:].argsort()
    temp[i,:]=temp[i,:][arrIndex[::-1]]
    index_all[i,:]=arrIndex
print(temp.shape)#5*5
print(index_all.shape)#5*5

all_term=5#Get the first 5 decomposition
c_all=np.zeros((c.shape[0],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2]))
for i in range(c.shape[0]):
    for u in range(all_term):
        c_all[i,:,:]=c_all[i,:,:]+c[i,:,:]+index_all[i,u]*temp[i,u]
print(c_all.shape)#(5,7,7)

#Step4:Assign it to particular position:
#Each timestamp from the same batch share the same Adj.
#MeanMI
data=[]
for j in range(a_all.shape[0]):
    for i in range(Term1):
        data.append(a_all[j,:,:])
data=np.array(data)
test=data
DyConcatMI1=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI1.shape)#(473,7,7)

data=[]
for j in range(b_all.shape[0]):
    for i in range(Term2):
        data.append(b_all[j,:,:])
data=np.array(data)
test=data
DyConcatMI2=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI2.shape)#(473,7,7)

data=[]
for j in range(c_all.shape[0]):
    for i in range(Term3):
        data.append(c_all[j,:,:])
data=np.array(data)
test=data
DyConcatMI3=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI3.shape)#(473,7,7)

# Step5: Add all of the adjacency matrices
DyConcatFFTeachMI=DyConcatMI1+DyConcatMI2+DyConcatMI3
print(DyConcatFFTeachMI.shape)#(473,7,7)

# Step6: Normalize adjacency matrices
from sklearn.preprocessing import MinMaxScaler
for i in range(DyConcatFFTeachMI.shape[0]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    DyConcatFFTeachMI[i,:,:] = scaler.fit_transform(DyConcatFFTeachMI[i,:,:])
print(DyConcatFFTeachMI)


(473, 3, 7)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(24, 7, 7)
(7, 7, 12)
(12, 7, 7)
(7, 7, 5)
(5, 7, 7)
(24, 24)
(24, 24)
(24, 7, 7)
(12, 12)
(12, 12)
(12, 7, 7)
(5, 5)
(5, 5)
(5, 7, 7)
(473, 7, 7)
(473, 7, 7)
(473, 7, 7)
(473, 7, 7)
[[[1.         0.25800582 0.22132181 ... 0.32850394 0.11149538 0.35670379]
  [0.         1.         0.18491694 ... 0.         0.15972734 0.        ]
  [0.17680435 0.40193684 1.         ... 0.12119995 0.17652692 0.25527595]
  ...
  [0.167177   0.07727409 0.         ... 1.         0.20144598 0.34736167]
  [0.01141662 0.31566915 0.11807075 ... 0.25878442 1.         0.11841477]
  [0.22805771 0.         0.13821908 ... 0.37487229 0.         1.        ]]

 [[1.         0.25800582 0.22132181 ... 0.32850394 0.11149538 0.35670379]
  [0.         1.         0.18491694 ... 0.         0.15972734 0.        ]
  [0.17680435 0.40193684 1.         ... 0.12119995 0.17652692 0.25527595]
  ...
  [0.167177   0.07727409 0.         ... 1.         0.20144598 0.34736167]
  [0.01141662 0.31566915 0.11807075 ... 0.25878442 1.         0

# Begin with DAMR

In [20]:
##Begin with DAMR
print(COVID_features_aftslid.shape)#473*3*7
print(COVID_X1_all_aftslid.shape)#473*7*30
print(COVID_Y1_all.shape)#(473, 7, 1)
print(Adj_dist.shape)#7*7
# Set Hyperpaprameters
tf.keras.backend.clear_session()#clear all
tf.autograph.set_verbosity(0)#reset to zero
learning_rate = 0.0001
batch_size =24
epochs = 100
seed = 42
verbose = 1

(473, 3, 7)
(470, 7, 30)
(473, 7, 1)
(7, 7)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
## Define Model---GAT+GCN+DIFFCONV
inputs_ret = Input(shape=(COVID_X1_all_aftslid.shape[1], COVID_X1_all_aftslid.shape[2], ))#7*30=node*lags(Time to look back)
inputs_feat = Input(shape=(COVID_features_aftslid.shape[2], COVID_features_aftslid.shape[1], ))#7*6=Node*feature
inputs_adj = Input(shape=(Adj_dist.shape[1],Adj_dist.shape[1], ))#7*7=node*node
print(inputs_ret.shape)
print(inputs_feat.shape)
print(inputs_adj.shape)

(None, 7, 30)
(None, 7, 3)
(None, 7, 7)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
#GATinput:1.input nodes*lags 2.adj:nodes*nodes
#GAToutput:1.channels*att_heads2.Att_weights=nodes*heads*nodes
GAT_output_P, Att_weights_P =  GATConv(16, attn_heads=4, concat_heads=True, dropout_rate=0.3, return_attn_coef=True, 
                                   activation='relu', use_bias=False)([inputs_ret, inputs_adj])

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
Att_weights_P = tf.math.reduce_mean(Att_weights_P, axis=2)#reduce heads,it will be nodes*nodes
print(Att_weights_P)

KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7), dtype=tf.float32, name=None), name='tf.math.reduce_mean/Mean:0', description="created by layer 'tf.math.reduce_mean'")


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
GCN_output_P =  GCNConv(16, dropout_rate=0.3, 
                        activation='relu', use_bias=True)([inputs_feat, inputs_adj])

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
diff_out = DiffusionConv(16, K=6, activation='tanh')([inputs_feat, Att_weights_P])#diff_out=nodes*channels

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
con_out_P_2 = Concatenate(axis=-1)([GAT_output_P, GCN_output_P,diff_out])
dense_P = Dense(8, activation='relu')(con_out_P_2)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
dense_out = Dropout(0.2)(dense_P)
outputs = Dense(1)(dense_out)



/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
model = Model(inputs=[inputs_ret, inputs_feat, inputs_adj], outputs=outputs)
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=["mae", "mape"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 7, 30)]      0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 7, 7)]       0                                            
__________________________________________________________________________________________________
gat_conv (GATConv)              ((None, 7, 64), (Non 2048        input_1[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 7, 3)]       0                                        

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
#Get 473*7*7 for static adjacency matrix
#3.1 Adj_dist
Adj_dist=adj_dist(COVID_lat,COVID_long)#7*7
Adj_dist=pd.DataFrame(Adj_dist)
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=Adj_dist
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_dist)
data=np.array(data)
Adj_dist=data
#3.2 MeanMI
Adj_MeanMI=MeanMI
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=MeanMI
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_MeanMI)
data=np.array(data)
Adj_MeanMI=data
#3.3 ConcatMI
Adj_ConcatMI=ConcatMI
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=Adj_ConcatMI
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_ConcatMI)
data=np.array(data)
Adj_ConcatMI=data
#3.4 FeatureMI
Adj_FeatureMI=FeatureMI
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=Adj_FeatureMI
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_FeatureMI)
data=np.array(data)
Adj_FeatureMI=data
# #3.9 ->MeanMI approach(3.1+3.2+3.5+3.8)
# coe1=0.1
# coe2=0.8
# coe3=0.4
# coe4=0.1
# Adj_all_MeanMI=coe1*Adj_dist+coe2*Adj_MeanMI+coe3*DyMeanMI+coe4*DyConcatFFTeachMI
# scaler = MinMaxScaler(feature_range=(-1, 1))
# for i in range(Adj_all_MeanMI.shape[0]):
#     Adj_all_MeanMI[i,:,:]=scaler.fit_transform(Adj_all_MeanMI[i,:,:])
# #3.10 ->()ConcatMI approach
# Adj_all_ConcatMI=coe1*Adj_dist+coe2*Adj_MeanMI+coe3*DyMeanMI+coe4*DyConcatFFTeachMI
# for i in range(Adj_all_ConcatMI.shape[0]):
#     Adj_all_ConcatMI[i,:,:]=scaler.fit_transform(Adj_all_ConcatMI[i,:,:])

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
scaler = MinMaxScaler(feature_range=(0, 1))

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
# #DyMeanMI
# #3.2 Mean MI-based dynamic adjacency matrix
# print(COVID_features_aftslid.shape)#(473, 3, 7)
# #Index all of the rows and columns
# MeanMI=[]
# for k in range(COVID_features_aftslid.shape[0]):
#     for i in range(COVID_features_aftslid.shape[2]):
#         for j in range(COVID_features_aftslid.shape[2]):
#             MeanMI.append(Global_MeanMI(COVID_features_aftslid[:k,:,i],
#                                     COVID_features_aftslid[:k,:,j],2))
# DyMeanMI=np.array(MeanMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[0])
# DyMeanMI=DyMeanMI.transpose(2, 0, 1)
# print(DyMeanMI.shape)#(473,7,7)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:

print(COVID_features_aftslid.shape)#473*3*7
COVID_features_aftslid= np.transpose(COVID_features_aftslid,(0,2,1))#473*7*3
print(COVID_X1_all_aftslid.shape)#473*7*30
print(np.array(COVID_Y1_all).shape)#(473, 7, 1)
print(np.array(Adj_MeanMI).shape)#473*7*7
#Normalization
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled1 = scaler.fit_transform(COVID_Y1_all[:,:,0])
COVID_Y1_all[:,:,0]=scaled1


(473, 3, 7)
(470, 7, 30)
(473, 7, 1)
(473, 7, 7)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
tf.keras.backend.clear_session()#clear all
tf.autograph.set_verbosity(0)#reset to zero
learning_rate = 0.0001
batch_size =72
epochs = 100
seed = 42
verbose = 1
ChangeAdj=Adj_FeatureMI
x_train=COVID_X1_all_aftslid[:int(len(COVID_X1_all_aftslid)*0.8),:,:]
x_val=COVID_X1_all_aftslid[int(len(COVID_X1_all_aftslid)*0.8):int(len(COVID_X1_all_aftslid)*0.9),:,:]
x_test=COVID_X1_all_aftslid[int(len(COVID_X1_all_aftslid)*0.9):,:,:]
features_train=COVID_features_aftslid[:int(len(COVID_features_aftslid)*0.8),:,:]
features_val=COVID_features_aftslid[int(len(COVID_features_aftslid)*0.8):int(len(COVID_features_aftslid)*0.9),:,:]
features_test=COVID_features_aftslid[int(len(COVID_features_aftslid)*0.9):,:,:]
adj_train=ChangeAdj[:int(len(ChangeAdj)*0.8),:,:]
adj_val=ChangeAdj[int(len(ChangeAdj)*0.8):int(len(ChangeAdj)*0.9),:,:]
adj_test=ChangeAdj[int(len(ChangeAdj)*0.9):,:,:]
y_train=COVID_Y1_all[:int(len(COVID_Y1_all)*0.8),:,:]
y_val=COVID_Y1_all[int(len(COVID_Y1_all)*0.8):int(len(COVID_Y1_all)*0.9),:,:]
y_test=COVID_Y1_all[int(len(COVID_Y1_all)*0.9):,:,:]

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
print(x_train.shape)
print(features_train.shape)
print(adj_train.shape)
print(y_train.shape)

(376, 7, 30)
(378, 7, 3)
(378, 7, 7)
(378, 7, 1)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
tf.config.experimental_run_functions_eagerly(True)
# Set Hyperpaprameters
tf.keras.backend.clear_session()#clear all
tf.autograph.set_verbosity(0)#reset to zero


# hists = model.fit(x= [x_train, features_train, adj_train],
#                   y= y_train, verbose=verbose, epochs=500,
#                   batch_size=batch_size, #validation_split=0.1, 
#                   validation_data=([x_val, features_val, adj_val], y_val),
#                   #test_data=([x_test, features_test, adj_test], y_test),
#   )

##If you want to train faster, try earlystopping.

hists = model.fit(x= [x_train, features_train, adj_train],
                  y= y_train, verbose=verbose, epochs=500,
                  batch_size=batch_size, #validation_split=0.1, 
                  validation_data=([x_val, features_val, adj_val], y_val),
                  #test_data=([x_test, features_test, adj_test], y_test),
                    callbacks=[EarlyStopping(#monitor="val_mean_absolute_error", 
                      monitor="val_loss", patience=20, restore_best_weights=True)],
  )


Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ValueError: Data cardinality is ambiguous:
  x sizes: 376, 378, 378
  y sizes: 378
Make sure all arrays contain the same number of samples.

In [ ]:
def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true))

def rmse(y_true, y_pred):
    return np.sqrt(np.mean(np.square(y_pred - y_true)))

def mape(y_true, y_pred, threshold=0.1):
    v = np.clip(np.abs(y_true), threshold, None)
    diff = np.abs((y_true - y_pred) / v)
    return np.mean(diff, axis=-1).mean()
###Write the function of mre
def mre(y_true, y_pred):
    return np.sum(np.abs(y_pred - y_true))/np.sum(np.abs(y_true))
def get_metrics(y, yp):
    return {

        "mae": np.round(mae(y, yp), 4),
        "mape": np.round(mape(y, yp),4),
        "mre": np.round(mre(y, yp),4),
        "rmse": np.round(rmse(y, yp), 4),
        #"MPE": np.round(MPE(y, yp), 4),
        #"R2": np.round(R_squared(y, yp),4)   
    }

# scaled1 = scaler.fit_transform(MeanMI)
#Get the tensor of test set after masking it.
test_predicted = model.predict([x_test, features_test, adj_test])
get_metrics(y_test.flatten(), test_predicted.flatten())


In [ ]:
#Get the mask of y_test.
Mask_BZ=pd.read_csv("Data/COVID/10%/BZ_10.csv")
Mask_CL=pd.read_csv("Data/COVID/10%/CL_10.csv")
Mask_ID=pd.read_csv("Data/COVID/10%/ID_10.csv")
Mask_IR=pd.read_csv("Data/COVID/10%/IR_10.csv")
Mask_MX=pd.read_csv("Data/COVID/10%/MX_10.csv")
Mask_PR=pd.read_csv("Data/COVID/10%/PR_10.csv")
Mask_SA=pd.read_csv("Data/COVID/10%/SA_10.csv")

In [ ]:
Mask_BZ=np.array(Mask_BZ)
missing_mask=[]
for i in range(Mask_BZ.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_BZ[i])))
missing_mask=np.array(missing_mask).reshape(Mask_BZ.shape[0],Mask_BZ.shape[1])
missing_mask_BZ_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of BZ
missing_mask_BZ_test1=missing_mask_BZ_test[:,0]

In [ ]:
Mask_CL=np.array(Mask_CL)
missing_mask=[]
for i in range(Mask_CL.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_CL[i])))
missing_mask=np.array(missing_mask).reshape(Mask_CL.shape[0],Mask_CL.shape[1])
missing_mask_CL_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of CL
missing_mask_CL_test1=missing_mask_CL_test[:,0]
Mask_ID=np.array(Mask_ID)
missing_mask=[]
for i in range(Mask_ID.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_ID[i])))
missing_mask=np.array(missing_mask).reshape(Mask_ID.shape[0],Mask_ID.shape[1])
missing_mask_ID_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of ID
missing_mask_ID_test1=missing_mask_ID_test[:,0]
Mask_IR=np.array(Mask_IR)
missing_mask=[]
for i in range(Mask_IR.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_IR[i])))
missing_mask=np.array(missing_mask).reshape(Mask_IR.shape[0],Mask_IR.shape[1])
missing_mask_IR_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of IR
missing_mask_IR_test1=missing_mask_IR_test[:,0]
Mask_MX=np.array(Mask_MX)
missing_mask=[]
for i in range(Mask_MX.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_MX[i])))
missing_mask=np.array(missing_mask).reshape(Mask_MX.shape[0],Mask_MX.shape[1])
missing_mask_MX_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of MX
missing_mask_MX_test1=missing_mask_MX_test[:,0]
Mask_PR=np.array(Mask_PR)
missing_mask=[]
for i in range(Mask_PR.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_PR[i])))
missing_mask=np.array(missing_mask).reshape(Mask_PR.shape[0],Mask_PR.shape[1])
missing_mask_PR_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of PR
missing_mask_PR_test1=missing_mask_PR_test[:,0]
Mask_SA=np.array(Mask_SA)
missing_mask=[]
for i in range(Mask_SA.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_SA[i])))
missing_mask=np.array(missing_mask).reshape(Mask_SA.shape[0],Mask_SA.shape[1])
missing_mask_SA_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of SA
missing_mask_SA_test1=missing_mask_SA_test[:,0]

In [ ]:
missing_mask_ALL_test1=np.stack((missing_mask_BZ_test1,missing_mask_CL_test1
            ,missing_mask_ID_test1,missing_mask_IR_test1,
            missing_mask_MX_test1,missing_mask_PR_test1,missing_mask_SA_test1
           ),axis=1)

In [ ]:
#Get test set result.
get_metrics(y_test[missing_mask_ALL_test1].flatten(), test_predicted[missing_mask_ALL_test1].flatten())